# Advanced execution

## Plugins

## Caching results

See [Caching and hashing](../explanation/hashing-caching.html) for more details.

## Environments (containers)

See [Containers and Environments](../explanation/environments.rst) for more details.

## Provenance